In [2]:
import os
import time

import numpy as np
from corner import corner
from matplotlib import pyplot as plt

import sbibm
import logging
logging.getLogger().setLevel(logging.INFO)


def corner_plot(gt_samples, algo_samples, true_params=None, title=None, txt=None, save_as=None, dpi=300):
    if true_params is not None:
        assert len(true_params) == gt_samples.shape[1], "true_params dimension mismatch with samples"

    fig = corner(
        gt_samples,
        color="tab:orange",
        hist_kwargs={"density": True},
    )
    corner(
        algo_samples,
        fig=fig,
        color="tab:blue",
        contour_kwargs=dict(linestyles="dashed"),
        hist_kwargs={"density": True},
    )

    # Add vertical lines for true parameters
    axes = fig.get_axes()
    num_params = gt_samples.shape[1]

    if true_params is not None:
        for i in range(num_params):
            ax = axes[i * (num_params + 1)]  # Get diagonal subplots
            ax.axvline(true_params[i], color="red", linestyle="dotted", linewidth=2, label="True Value")

    # Adjust appearance
    for ax in axes:
        ax.tick_params(axis="both", labelsize=12)

    # Add legend
    lgd = fig.legend(
        labels=["Ground truth", "Algo", "True Value"] if true_params is not None else ["Ground truth", "Algo"],
        loc="upper center",
        bbox_to_anchor=(0.8, 0.6),
    )
    fig.suptitle(title, fontsize=20)
    fig.tight_layout()
    if txt is not None:
        text_art = fig.text(
            0.0, -0.10, txt, wrap=True, horizontalalignment="left", fontsize=12
        )
        extra_artists = (text_art, lgd)
    else:
        extra_artists = (lgd,)
    if save_as is not None:
        fig.savefig(
            save_as,
            dpi=dpi,
            bbox_extra_artists=extra_artists,
            bbox_inches="tight",
        )
    return fig



RESULTS_DIR = "./experiment_results"



In [ ]:
# Instantiate a task
task = sbibm.get_task("svar")
# task = sbibm.get_task("bernoulli_glm")  # See sbibm.get_available_tasks() for all tasks
# task = sbibm.get_task("sir")
# task = sbibm.get_task("gaussian_mixture")
# task = sbibm.get_task("gaussion_linear_uniform")

In [ ]:
from sbibm.algorithms import pyvbmc,snpe,bolfi
# run pyvbmc with exact likelihood
posterior_samples, _, _ = pyvbmc(task=task, num_samples=10000, num_observation=1, noisy_likelihood=False)

# run pyvbmc with noisy likelihood
posterior_samples, total_simulations, _ = pyvbmc(task=task, num_samples=10000, num_observation=1, simulations_per_eval=100) # run until converge
posterior_samples, _, _ = pyvbmc(task=task, num_samples=10000, num_observation=1, num_simulations=10000) # run with fixed simulation budget


# run snpe
posterior_samples, _, _ = snpe(task=task, num_samples=10000, num_observation=1, num_simulations=2000)

# run bolfi
posterior_samples, _, _ = bolfi(task=task, num_samples=10000, num_observation=1, num_simulations=200)


In [ ]:
# visualize inferred vs reference posterior distribution 
ref_samples = task.get_reference_posterior_samples(num_observation=1)
true_params = task.get_true_parameters(num_observation=1)

corner_plot(ref_samples.numpy(),posterior_samples.numpy(), true_params.squeeze().numpy())
